## line格式检查

In [59]:
import time
# source_path = "/world/data-c26/liliang/person_attribute/v5/lst/age/source/v5_anta3_11-25.json"
source_path = "/world/data-c9/dl-data/587708b9603073b127434c32/5d82e88ca07f176fa47b9e27/15688603000730.9526127707718031_list.json"
with open(source_path, "r") as f:
    for line in f:
#         line = line.replace("\t", " ")
#         print(line.strip())
        print(line.split(" "))
        time.sleep(3)

['/world/data-c22/train-data/xiaoxianli/bi/chongqing_anta827-910/31-40label/1566866407_24643467.jpg\t5\t5d78a7db80f3797481a7477f\n']
['/world/data-c22/train-data/xiaoxianli/bi/chongqing_anta827-910/31-40label/1566866407_43079797.jpg\t5\t5d78a7db80f3797481a74780\n']


KeyboardInterrupt: 

## tab 转空格（跑完keypoints之后用）

In [60]:
from subprocess import call
ori_path = "/world/data-c26/liliang/person_attribute/v5/lst/age/source/v5_anta4_31-40.json"
new_path = "/world/data-c26/liliang/person_attribute/v5/lst/age/source/v5_anta3_31-40_sp.json"
new_f = open(new_path, "w")
cnt = 0
with open(ori_path, "r") as f:
    for line in f:
        if len(line)<40:
            print(line)
            continue
        line = line.replace("\t", " ")
        new_f.write(line)
        cnt += 1
    print("finish, total: %s"%cnt)
new_f.close()
call(["rm", ori_path]) 
call(["mv", new_path, ori_path])

finish, total: 73827


0

## json筛选 （先根据原有分布选择upper_bound的值）

In [81]:
import time
import os
def select_json():
    task = "age"
    source_path = "/world/data-c26/liliang/person_attribute/v5/lst/%s/source/v5_total_add.json"%task
    out_path = os.path.join(os.path.dirname(source_path), "v5_select.json")
    recs = open(source_path, "r").readlines()
    age_mapping = {0:0, 1:1, 2:2, 3:3, 4:4, 5:5, 12:6, 6:7, 7:8, 8:9, 9:10, 10:11}
#     upper_bound = {3: 20000, 4: 20000, 5:20000, 6:17741, 7:17741}
    upper_bound = {0:0, 1:0, 2:0, 3:0, 4:800, 5:800, 6:0, 7:0, 8:0, 9:0, 10:0, 11:0}
    cnt_dict = {0:0, 1:0, 2:0, 3:0, 4:0, 5:0, 6:0, 7:0, 8:0, 9:0, 10:0, 11:0}

    with open(out_path, "w") as out_f:
        for line in recs:
            path, label, keypoints = line.split(" ")
            label = int(label)
            if label in [11, 13]:continue
            label = age_mapping[label]
            if label in upper_bound.keys() and cnt_dict[label]>upper_bound[label]:
                continue
            cnt_dict[label]+=1
            out_f.write(line)
    print(cnt_dict)
    print("finish selecting")

select_json()

{0: 1, 1: 1, 2: 1, 3: 1, 4: 801, 5: 801, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1, 11: 1}
finish selecting


## 拼接json（两个version）

In [82]:
import os
def json_concat():
    task = "age"
    base_path = "/world/data-c26/liliang/person_attribute/v5/lst/%s"%task
    last_version_path = os.path.join(base_path, "source/train_keypoints_before_v5.json")
    this_version_path_1 = os.path.join(base_path, "source/train_keypoints_v5.json")
    this_version_path_2 = os.path.join(base_path, "source/v5_anta1_11-25_filtercam.json")
    this_version_path_3 = os.path.join(base_path, "source/v5_anta2_11-25_filtercam.json")
    this_version_path_4 = os.path.join(base_path, "source/v5_anta3_11-25_filtercam.json")
    this_version_path_5 = os.path.join(base_path, "source/v5_anta4_31-40_filtercam.json")
    out_json_path = os.path.join(base_path, "train_keypoints.json")

    ## for age distribution select
    this_version_path_6 = os.path.join(base_path, "source/v5_select.json")
    out_json_path = os.path.join("/world/data-c26/liliang/person_attribute/v5/lst/age_test_8h8h", "train_keypoints.json")
#     out_json_path = os.path.join(base_path, "source/v5_total_add.json")

    file_paths_tobe_concat = [last_version_path, this_version_path_6]
    total_rec = []

    for fp in file_paths_tobe_concat:
        rec = open(fp, "r").readlines()
        total_rec += rec
        print("%s: %s records"%(fp.split("/")[-1], len(rec)))

    with open(out_json_path, "w") as out_f:
        for line in total_rec:
            out_f.write(line)

    print("[%s] finish concating, total: %s records"%(task, len(total_rec)))

json_concat()

train_keypoints_before_v5.json: 573532 records
v5_select.json: 1612 records
[age] finish concating, total: 575144 records


## 去除line中的cid（如有必要，跑完keypoints之后用, tab转空格之后）

In [61]:
from subprocess import call
def filter_cid():
    task = "age"
    source_path = "/world/data-c26/liliang/person_attribute/v5/lst/%s/source/v5_anta4_31-40.json"%task
    filter_path = source_path.split(".")[0] + "_filtercam.json"

    recs = open(source_path, "r").readlines()

    with open(filter_path, "w") as out_f:
        for line in recs:
            path, label, cid, keypoints = line.split(" ")
            new_line = path + " " + label + " " + keypoints
            out_f.write(new_line)

    print("finish filtering")
    call(["rm", source_path]) 

filter_cid()

finish filtering


## 备份导出数据，改json中图片路径

In [44]:
import os
from subprocess import call
import ipdb

base_path = "/world/data-c26/liliang/anta_json"
paths = ["lehuameilan"]
for p in paths:
    this_path = os.path.join(base_path, p) + ".json"
    chg_path = os.path.join(base_path, p) + "_new.json"
    recs = open(this_path, "r").readlines()
    with open(chg_path, "w") as out_f:
        for line in recs:
            path, infos = line.split("\t")
            filename = path.split("/")[-1]
            new_path = "/world/data-c40/liliang/" + p + "/" + filename
            new_line = new_path + "\t" + infos
            out_f.write(new_line)